<a href="https://colab.research.google.com/github/rojobravo/tensorflow-2-public/blob/main/C3_Data-pipelines/W1/assignment/myversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from os import getcwd

import tensorflow as tf
import tensorflow_datasets as tfds

In [35]:
def my_one_hot(feature, label):
    """
    Encode the labels to one-hot using tf.one_hot with depth equal to total
    number of classes here which are rock, paper and scissors
    """
    # Ensure label is an integer
    label = tf.cast(label, tf.int32)
    # Create one-hot encoded labels with the correct depth
    one_hot_labels = tf.one_hot(label, depth=3)
    # Squeeze the labels to remove the extra dimension if exists
    #one_hot_labels = tf.squeeze(one_hot_labels)

    return feature, one_hot_labels

In [36]:
filePath = f"{getcwd()}/data"

train_data = tfds.load(name='rock_paper_scissors',data_dir=filePath, split='train', as_supervised=True)# YOUR CODE HERE)
val_data = tfds.load(name='rock_paper_scissors',data_dir=filePath, split="test", as_supervised=True)# YOUR CODE HERE)

# Testing train_data and val_data if loaded correctly

train_data_len = len(list(train_data))
val_data_len = len(list(val_data))

print(train_data_len)
print(val_data_len)

2520
372


In [37]:
# prompt: one-hot encode the train and validation labels using the function you defined earlier

train_data = train_data.map(my_one_hot)
val_data = val_data.map(my_one_hot)

print(type(train_data))

<class 'tensorflow.python.data.ops.map_op._MapDataset'>


In [38]:
_,info = tfds.load(name='rock_paper_scissors',data_dir=filePath, split='train', as_supervised=True, with_info = True)

# DO NOT EDIT THIS
print(info.features['image'].shape)

(300, 300, 3)


In [39]:
train_batches = train_data.shuffle(100).batch(10)
validation_batches = val_data.batch(32)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape= info.features['image'].shape),# YOUR CODE HERE),
    # YOUR CODE HERE - Add a maxpool2d layer with kernel (2,2)
    tf.keras.layers.MaxPool2D((2,2)),
    # YOUR CODE HERE - Add a flatten layer
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(3, activation='softmax')# YOUR CODE HERE)  # Remember there are 3 classes to classify and to use proper activation
])

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 298, 298, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 149, 149, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 355216)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │       1,065,651 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,066,099 (4.07 MB)

 Trainable params: 1,066,099 (4.07 MB)

 Non-trainable params: 0 (0.00 B)

In [40]:
EPOCH = 3

In [43]:
# prompt: create the model.compile using 'adam'

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [44]:
history = model.fit(train_batches, epochs= EPOCH , validation_data=validation_batches, validation_steps=1)

print("Final Training Accuracy:-",history.history['accuracy'][-1])
print("Final Validation Accuracy:-",history.history['val_accuracy'][-1])

Epoch 1/3
252/252 ━━━━━━━━━━━━━━━━━━━━ 105s 415ms/step - accuracy: 0.5824 - loss: 2848.6917 - val_accuracy: 0.5938 - val_loss: 98.4770
Epoch 2/3
252/252 ━━━━━━━━━━━━━━━━━━━━ 105s 416ms/step - accuracy: 0.9734 - loss: 2.9617 - val_accuracy: 0.7188 - val_loss: 43.8084
Epoch 3/3
252/252 ━━━━━━━━━━━━━━━━━━━━ 141s 414ms/step - accuracy: 0.9814 - loss: 1.0357 - val_accuracy: 0.6875 - val_loss: 43.2093
Final Training Accuracy:- 0.9892857074737549
Final Validation Accuracy:- 0.6875
